# Clustering on the Soil Moisture Dataset

Soil Moisture Dataset

Libraries we need are imported first

In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
import xarray as xr
from scipy.interpolate import griddata
from scipy import interpolate
import statsmodels.api as sm

Data used in this exercise is stored in the DATA folder

In [30]:
thefile = '../../../DATA/TPdata_199301'

The dataframe is created using the .read_csv function of Pandas

In [33]:
data = pd.read_csv(thefile, sep='\t', header=None)

The original CSV file is structured as a dataframe with the corresponding categories

In [36]:
data.columns = ["cellNr", "latitude", "longitude", "backscatter","emissivity_v","emissivity_h","ts_amplitude","ndvi","lmd_soilWetness"]

In [38]:
data = data.replace('   NaN',pd.NA)

In [40]:
data['backscatter'] = pd.to_numeric(data['backscatter'])
data['emissivity_v'] = pd.to_numeric(data['emissivity_v'])
data['emissivity_h'] = pd.to_numeric(data['emissivity_h'])
data['ts_amplitude'] = pd.to_numeric(data['ts_amplitude'])
data['ndvi'] = pd.to_numeric(data['ndvi'])
data['lmd_soilWetness'] = pd.to_numeric(data['lmd_soilWetness'])

In [42]:
data = data.dropna()

## Clustering